Using CNN

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from PIL import Image
from tqdm.auto import tqdm

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
label_mapping = {
    '000': 0,
    '001': 1,
    '010': 2,
    '011': 3,
    '100': 4,
    '101': 5,
    '110': 6,
    '111': 7
}

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Colab Notebooks/AI_Project/인지프 팀플/classification/data

/content/drive/MyDrive/Colab Notebooks/AI_Project/인지프 팀플/classification/data


In [5]:
df = pd.read_excel('label2.xlsx')
# Filter out the rows with '-' in their labels.
df = df[~df['label'].str.contains("-")]

In [6]:
print(df)

      File Number life head  heart label
0               1    0    0      0   000
1               2    0    0      0   000
2               5    1    1      0   110
3              12    0    0      1   001
4              13    1    1      0   110
...           ...  ...  ...    ...   ...
1030         5353    0    0      1   001
1031         5354    1    0      1   101
1032         5366    1    0      0   100
1033         5367    0    0      0   000
1035         5385    0    0      1   001

[965 rows x 5 columns]


In [7]:
# Convert labels to categorical values.
df['label'] = df['label'].astype(str)

In [8]:
# Split the dataset into training and testing sets.
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [9]:
# Pre-process the images and load them into memory.
def load_images(image_df):
    images = []
    for idx, row in tqdm(image_df.iterrows(), total=image_df.shape[0]):
        image_path = os.path.join('./Mask', f"image{row['File Number']}.png")
        image = Image.open(image_path)
        image = image.resize((128, 128))  # Resize images to 128x128
        if image.mode != 'L':
            image = image.convert('L')  # Convert to grayscale if not already
        image = np.array(image) / 255.0  # Normalize pixel values to [0, 1]
        images.append(image)
    return np.array(images).reshape(-1, 128, 128, 1)  # Reshape for the CNN

train_images = load_images(train_df)
test_images = load_images(test_df)

  0%|          | 0/772 [00:00<?, ?it/s]

  0%|          | 0/193 [00:00<?, ?it/s]

In [10]:
train_df['label'] = train_df['label'].map(label_mapping)
test_df['label'] = test_df['label'].map(label_mapping)

In [11]:
train_labels = to_categorical(train_df['label'], num_classes=8)
test_labels = to_categorical(test_df['label'], num_classes=8)

In [12]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(8, activation='softmax')  # 8 classes
])

In [13]:
# Compile the model.
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
# Train the model.
history = model.fit(train_images, train_labels, epochs=10, validation_split=0.1)

Epoch 1/10
22/22 [==============================] - 24s 1s/step - loss: 1.9878 - accuracy: 0.2205 - val_loss: 1.8963 - val_accuracy: 0.2692
Epoch 2/10
22/22 [==============================] - 25s 1s/step - loss: 1.8714 - accuracy: 0.2882 - val_loss: 1.8333 - val_accuracy: 0.2949
Epoch 3/10
22/22 [==============================] - 23s 1s/step - loss: 1.8205 - accuracy: 0.3285 - val_loss: 1.7557 - val_accuracy: 0.3462
Epoch 4/10
22/22 [==============================] - 22s 1s/step - loss: 1.6572 - accuracy: 0.3905 - val_loss: 1.6969 - val_accuracy: 0.3590
Epoch 5/10
22/22 [==============================] - 23s 1s/step - loss: 1.5368 - accuracy: 0.4352 - val_loss: 1.6718 - val_accuracy: 0.4231
Epoch 6/10
22/22 [==============================] - 25s 1s/step - loss: 1.3632 - accuracy: 0.5014 - val_loss: 1.5761 - val_accuracy: 0.4487
Epoch 7/10
22/22 [==============================] - 22s 1s/step - loss: 1.1542 - accuracy: 0.5793 - val_loss: 1.4930 - val_accuracy: 0.5513
Epoch 8/10
22/22 [==

In [15]:
# Evaluate the model on the test set.
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

7/7 [==============================] - 2s 207ms/step - loss: 1.9096 - accuracy: 0.4197
Test accuracy: 41.97%
